In [45]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
import copy
sc.stop()
sc = SparkContext('local')
spark = SparkSession(sc)

In [46]:
import os,glob
path0 = 'tsvfiles/'
df = spark.read.csv(
    os.path.join(path0,"SUNM.tsv"), sep=r'\t', header=False, mode="DROPMALFORMED")

dataColl=df.collect()
for row in dataColl[0:5]:
    print(row[0] + "," +str(row[1]))

S0558,well i vote out and then when you you know when you've do it and er
S0517,do vote out
S0558,yeah
S0517,yeah
S0558,and i think have this feel you know on occasion i've have these feel and i think oh i dunno if i've do the right thing


In [47]:
import xmltodict
def extract_time(f):
    with open(f) as fd:
        doc = xmltodict.parse(fd.read())
    s_time = doc['text']['header']['rec_length']
    s_seconds = (int(s_time[0])*3600)+ (int(s_time[2])*10+int(s_time[3]))*60 + int(s_time[5])*10+int(s_time[6])
    return s_seconds

s = extract_time('S376.xml')
print(s)

2026


In [48]:
def extract_speakers_n(df1, n):
    dataC = df1.collect()
    speaker = [dataC[0][0]]
    for row in dataC:
        if row[0] not in speaker:
            speaker.append(row[0])
        if len(speaker) == n:
            break
    #rdd1 = rdd1.rdd
    return df1, speaker

In [49]:
def extract_dialogues_dic_n(rdd1, speaker):
    n = len(speaker)
    dataC = rdd1.collect()
    speaker_dic = [{} for x in range(0,n)]
    
    
    for i in range(0,len(dataC)):
        if dataC[i][0] in speaker:
            p = speaker.index(dataC[i][0])
            if dataC[i][1]!=None:
                speaker_dic[p][i] = dataC[i][1].split()
    
    return speaker, speaker_dic

In [50]:
def get_vo(speaker, speaker_dic):
    n = len(speaker)
    voand = []
    voor = []
    speaker_vo = [[] for x in range(0,n)]
    vo_group = {}
    token_n = 0
    
    for i in range(0,n):
        for k,tokens in speaker_dic[i].items():
            
            for token in tokens:
                token_n = token_n+1
                if token not in voor:
                    voor.append(token)
                if token not in speaker_vo[i]:
                    speaker_vo[i].append(token)
    
    countand = 0
    if n==2:
        for token0 in speaker_vo[0]:
            if token0 in speaker_vo[1]:
                countand = countand+1
    if n==3:
        for token0 in speaker_vo[0]:
            if (token0 in speaker_vo[1]) and (token0 in speaker_vo[2]):
                countand = countand+1
    if n==4:
        for token0 in speaker_vo[0]:
            if (token0 in speaker_vo[1]) and (token0 in speaker_vo[2]) and (token0 in speaker_vo[3]):
                countand = countand+1
    countor = len(voor)
    overlap = countand/countor
    vo_group = {"speaker_number": n, "vo_overlap": countand, "vo_total": countor, "total_token_number":token_n, "overlap": overlap} 
    return vo_group
            

In [51]:
def get_vo_pairwise(speaker, speaker_dic):
    n = len(speaker)
    voand = []
    voor = []
    speaker_vo = [[] for x in range(0,n)]
    vo_group = {}
    token_n = 0
    
    for i in range(0,n):
        for k,tokens in speaker_dic[i].items():
            for token in tokens:
                token_n = token_n+1
                if token not in voor:
                    voor.append(token)
                if token not in speaker_vo[i]:
                    speaker_vo[i].append(token)
    
    if n==3:
        countand_list = [[] for j in range(0,3)]
        x = 0
        for s_i in range(0,n-1):
            for s_j in range(s_i+1,n):
                for token in speaker_vo[s_i]:
                    if token in speaker_vo[s_j]:
                        countand_list[x].append(token)
                x = x+1
        countand = 0               
        for c in range(0,3):
            countand = countand + len(countand_list[c])
        
        countor = len(voor)
        overlap = countand/(3*countor)
        
        vo_group = {"speaker_number": n, "vo_overlap": countand, "vo_total": countor, "total_token_number":token_n, "overlap": overlap,\
                    "s1_and_s2": len(countand_list[0]), "s1_and_s3": len(countand_list[1]),"s2_and_s3": len(countand_list[2])}
            
            
    if n==4:
        countand_list = [[] for i in range(0,6)]
        x = 0
        for s_i in range(0,n-1):
            for s_j in range(s_i+1,n):
                for token in speaker_vo[s_i]:
                    if token in speaker_vo[s_j]:
                        countand_list[x].append(token)
                x = x+1
        countand = 0               
        for c in range(0,6):
            countand = countand + len(countand_list[c])
        countor = len(voor)
        overlap = countand/(6*countor)
        
        vo_group = {"speaker_number": n, "vo_overlap": countand, "vo_total": countor, "total_token_number":token_n, "overlap": overlap,\
                    "s1_and_s2": len(countand_list[0]), "s1_and_s3": len(countand_list[1]),"s1_and_s4": len(countand_list[2]),\
                   "s2_and_s3": len(countand_list[3]),"s2_and_s4": len(countand_list[4]),"s3_and_s4": len(countand_list[5])}
    
    
    return vo_group

In [52]:
def get_vo_average_speaker(speaker, speaker_dic):
    n = len(speaker)
    voand = []
    voor = []
    speaker_vo = [[] for x in range(0,n)]
    vo_group = {}
    token_n = 0
    
    for i in range(0,n):
        for k,tokens in speaker_dic[i].items():
            token_n = token_n + len(tokens)
            for token in tokens:
#                 token_n = token_n+1
                if token not in voor:
                    voor.append(token)
                if token not in speaker_vo[i]:
                    speaker_vo[i].append(token)
                    
    if n==2:
        voand_2 = []
        for s0 in speaker_vo[0]:
            if s0 in speaker_vo[1]:
                if s0 not in voand_2:
                    voand_2.append(s0)
                    
        countand = len(voand_2)
        countor = len(voor)
        overlap = countand/countor
        
        vo_group = {"speaker_number": n, "vo_overlap": countand, "vo_total": countor, "total_token_number":token_n, "overlap": overlap}
    
    if n==3:
        countand_list = [[] for j in range(0,3)]
        x = 0
        for s_i in range(0,n):
            for s_j in range(0,n):
                if s_j != s_i:
                    for token in speaker_vo[s_i]:
                        if token in speaker_vo[s_j]:
                            if token not in countand_list[x]:
                                countand_list[x].append(token)
            x = x+1
            
        countand = 0               
        for c in range(0,3):
            countand = countand + len(countand_list[c])
        
        countor = len(voor)
        overlap = countand/(3*countor)
        
        vo_group = {"speaker_number": n, "vo_overlap": countand, "vo_total": countor, "total_token_number":token_n, "overlap": overlap,\
                    "s1_and_other": len(countand_list[0]), "s2_and_other": len(countand_list[1]),"s3_and_other": len(countand_list[2])}
            
            
    if n==4:
        countand_list = [[] for i in range(0,4)]
        x = 0
        
        for s_i in range(0,n):
            for s_j in range(0,n):
                if s_j != s_i:
                    for token in speaker_vo[s_i]:
                        if token in speaker_vo[s_j]:
                            if token not in countand_list[x]:
                                countand_list[x].append(token)
            x = x+1
            
        countand = 0               
        for c in range(0,4):
            countand = countand + len(countand_list[c])
        
        countor = len(voor)
        overlap = countand/(4*countor)

        
        vo_group = {"speaker_number": n, "vo_overlap": countand, "vo_total": countor, "total_token_number":token_n, "overlap": overlap,\
                    "s1_and_other": len(countand_list[0]), "s2_and_other": len(countand_list[1]),"s3_and_other": len(countand_list[2]),\
                   "s4_and_other": len(countand_list[3])}
    
    
    return vo_group

In [53]:
def get_vo_average_speaker_in_window(speaker, speaker_dic, start, end):
    n = len(speaker)
    voand = []
    voor = []
    speaker_vo = [[] for x in range(0,n)]
    vo_group = {}
    token_n = 0
    
    for i in range(0,n):
        for k,tokens in speaker_dic[i].items():
            if k>=start and k<end:
                token_n = token_n + len(tokens)
                for token in tokens:
#                 token_n = token_n+1
                    if token not in voor:
                        voor.append(token)
                    if token not in speaker_vo[i]:
                        speaker_vo[i].append(token)
    if n==2:
        voand_2 = []
        for s0 in speaker_vo[0]:
            if s0 in speaker_vo[1]:
                if s0 not in voand_2:
                    voand_2.append(s0)
                    
        countand = len(voand_2)
        countor = len(voor)
        overlap = countand/countor
        
        vo_group = {"speaker_number": n, "vo_overlap": countand, "vo_total": countor, "total_token_number":token_n, "overlap": overlap, "begin":start, "end":end}
    
    
    if n==3:
        countand_list = [[] for j in range(0,3)]
        x = 0
        for s_i in range(0,n):
            for s_j in range(0,n):
                if s_j != s_i:
                    for token in speaker_vo[s_i]:
                        if token in speaker_vo[s_j]:
                            if token not in countand_list[x]:
                                countand_list[x].append(token)
            x = x+1
            
        countand = 0               
        for c in range(0,3):
            countand = countand + len(countand_list[c])
        
        countor = len(voor)
        overlap = countand/(3*countor)
        
        vo_group = {"speaker_number": n, "vo_overlap": countand, "vo_total": countor, "total_token_number":token_n, "overlap": overlap,\
                    "s1_and_other": len(countand_list[0]), "s2_and_other": len(countand_list[1]),"s3_and_other": len(countand_list[2]),\
                   "begin":start, "end":end}
            
            
    if n==4:
        countand_list = [[] for i in range(0,4)]
        x = 0
        
        for s_i in range(0,n):
            for s_j in range(0,n):
                if s_j != s_i:
                    for token in speaker_vo[s_i]:
                        if token in speaker_vo[s_j]:
                            if token not in countand_list[x]:
                                countand_list[x].append(token)
            x = x+1
            
        countand = 0               
        for c in range(0,4):
            countand = countand + len(countand_list[c])
        
        countor = len(voor)
        overlap = countand/(4*countor)

        
        vo_group = {"speaker_number": n, "vo_overlap": countand, "vo_total": countor, "total_token_number":token_n, "overlap": overlap,\
                    "s1_and_other": len(countand_list[0]), "s2_and_other": len(countand_list[1]),"s3_and_other": len(countand_list[2]),\
                   "s4_and_other": len(countand_list[3]), 'begin':start, 'end':end}
    
    
    return vo_group
    
    

In [54]:
def number_of_slot_n(speaker_dic,length_u):
    number_of_u = 0
    for s_d in speaker_dic:
        number_of_u = number_of_u + len(s_d)
    
    number_slot = int(number_of_u - number_of_u%length_u)/length_u
    return int(number_slot)

In [55]:
def number_of_total_u(speaker_dic):
    number_of_u = 0
    for s_d in speaker_dic:
        number_of_u = number_of_u + len(s_d)
    
    return number_of_u

In [56]:
def get_vo_average_speaker_temporal_n(speaker, speaker_dic, length):
    slot_number = number_of_slot_n(speaker_dic, length)
    vo_dic_list = []
    total_u = 0
    
    for s in speaker_dic:
        total_u = total_u + len(s)
    
    for i in range(0,slot_number):
        vo_dic = get_vo_average_speaker_in_window(speaker,speaker_dic, length*i, length*(i+1))
        vo_dic_list.append(vo_dic)
        
    if total_u > slot_number*length:
        vo_dic_end = get_vo_average_speaker_in_window(speaker, speaker_dic, slot_number*length, total_u)
        vo_dic_list.append(vo_dic_end)
    
    return vo_dic_list

In [57]:
out1 = extract_speakers_n(df,3)

In [58]:
out2 = extract_dialogues_dic_n(out1[0],out1[1])

In [59]:
out3 = get_vo(out2[0], out2[1])
print(out3)

{'speaker_number': 2, 'vo_overlap': 802, 'vo_total': 1857, 'total_token_number': 30732, 'overlap': 0.43187937533656434}


In [60]:
out4 = get_vo_pairwise(out2[0], out2[1])
print(out4)

{}


In [61]:
out5 = get_vo_average_speaker(out2[0], out2[1])
print(out5)

{'speaker_number': 2, 'vo_overlap': 802, 'vo_total': 1857, 'total_token_number': 30732, 'overlap': 0.43187937533656434}


In [62]:
out6 = get_vo_average_speaker_temporal_n(out2[0], out2[1], 50)
for o in out6:
    print(o['overlap'])

0.23225806451612904
0.34177215189873417
0.24444444444444444
0.19576719576719576
0.26875
0.27071823204419887
0.24550898203592814
0.22549019607843138
0.2621951219512195
0.2835820895522388
0.3048128342245989
0.25
0.30656934306569344
0.2536231884057971
0.2746478873239437
0.2140468227424749
0.12589928057553956
0.21348314606741572
0.19101123595505617
0.2795031055900621
0.18095238095238095
0.26842105263157895
0.21212121212121213
0.32061068702290074
0.27611940298507465
0.2192513368983957
0.25
0.20418848167539266
0.3548387096774194
0.08560311284046693
0.27710843373493976
0.2430939226519337
0.18627450980392157
0.2640449438202247
0.2222222222222222
0.27564102564102566
0.2733333333333333
0.2222222222222222
0.1853448275862069
0.2796208530805687
0.22807017543859648
0.25263157894736843
0.26277372262773724
0.10050251256281408
0.26732673267326734
0.3197278911564626
0.25654450261780104
0.3671875
0.2550335570469799
0.15517241379310345
0.27835051546391754
0.30113636363636365
0.25
0.2413793103448276
0.1436

In [63]:
import pandas as pd
dl = []
temporal_vo = []
import glob,os
path0 = 'tsvfiles/'
path = 'vo_calculate/vo_group2_temporal_average_speaker_in_window'
files = glob.glob(os.path.join(path0, "****.tsv"))
print(len(files))
for f in files:
    dff = spark.read.csv(f, sep=r'\t', header=False, mode="DROPMALFORMED")
    speaker_f = extract_speakers_n(dff,2)
    dialogue_dic_f = extract_dialogues_dic_n(speaker_f[0],speaker_f[1])
    df_vo_temporal = get_vo_average_speaker_temporal_n(dialogue_dic_f[0],dialogue_dic_f[1],50)
    for d in df_vo_temporal:
        d['file'] = f[len(f)-8:len(f)]
    temporal_vo = pd.DataFrame(df_vo_temporal)
    temporal_vo.to_csv(os.path.join(path, f[len(f)-8:len(f)-4]+'.csv'), index = False)

622


In [21]:
print(f)

tsvfiles-3/SW6K.tsv


In [23]:
dff = spark.read.csv(f, sep=r'\t', header=False, mode="DROPMALFORMED")
speaker_f = extract_speakers_n(dff,4)
dialogue_dic_f = extract_dialogues_dic_n(speaker_f[0],speaker_f[1])
print(number_of_total_u(dialogue_dic_f[1]))

2200


In [123]:
import pandas as pd
dl = []
import glob,os
path0 = 'tsvfiles-3/'
path = 'vo_calculate/'
files = glob.glob(os.path.join(path0, "****.tsv"))
print(files)
for f in files:
    dff = spark.read.csv(f, sep=r'\t', header=False, mode="DROPMALFORMED")
    speaker_f = extract_speakers_n(dff,3)
    dialogue_dic_f = extract_dialogues_dic_n(speaker_f[0],speaker_f[1])
    df_vo = get_vo_average_speaker(dialogue_dic_f[0],dialogue_dic_f[1])
    df_vo['file'] = f[len(f)-8:len(f)]
    dl.append(df_vo)

dl_s = pd.DataFrame(dl)
dl_s.to_csv(os.path.join(path,'vo_group_average_speaker_3.csv'), index = False)


['tsvfiles-3/SWHR.tsv', 'tsvfiles-3/SWSX.tsv', 'tsvfiles-3/SR4E.tsv', 'tsvfiles-3/SK82.tsv', 'tsvfiles-3/SFRW.tsv', 'tsvfiles-3/SDQ8.tsv', 'tsvfiles-3/SVLZ.tsv', 'tsvfiles-3/SX4N.tsv', 'tsvfiles-3/SV5R.tsv', 'tsvfiles-3/SHKZ.tsv', 'tsvfiles-3/SAAF.tsv', 'tsvfiles-3/SA88.tsv', 'tsvfiles-3/S8NX.tsv', 'tsvfiles-3/S8LG.tsv', 'tsvfiles-3/S49V.tsv', 'tsvfiles-3/S4NB.tsv', 'tsvfiles-3/SQP2.tsv', 'tsvfiles-3/SNQD.tsv', 'tsvfiles-3/SE7R.tsv', 'tsvfiles-3/SUJ8.tsv', 'tsvfiles-3/SPC3.tsv', 'tsvfiles-3/SEYD.tsv', 'tsvfiles-3/SGMT.tsv', 'tsvfiles-3/S73U.tsv', 'tsvfiles-3/S2NQ.tsv', 'tsvfiles-3/SPZR.tsv', 'tsvfiles-3/SJH6.tsv', 'tsvfiles-3/S6AZ.tsv', 'tsvfiles-3/SQ36.tsv', 'tsvfiles-3/SHHU.tsv', 'tsvfiles-3/S69V.tsv', 'tsvfiles-3/SVC7.tsv', 'tsvfiles-3/SJRX.tsv', 'tsvfiles-3/SH2P.tsv', 'tsvfiles-3/S8LS.tsv', 'tsvfiles-3/S8X7.tsv', 'tsvfiles-3/S457.tsv', 'tsvfiles-3/SJ2E.tsv', 'tsvfiles-3/STNE.tsv', 'tsvfiles-3/SPYH.tsv', 'tsvfiles-3/SP5V.tsv', 'tsvfiles-3/SN33.tsv', 'tsvfiles-3/S2ZU.tsv', 'tsvfiles-

KeyboardInterrupt: 